# **Trabajo Práctico 1: Clasificador de Recomendaciones Recreativas utilizando Procesamiento de Lenguaje Natural (NLP)**

**Año:** 2024  
**Materia:** Procesamiento del Lenguaje Natural  
**Integrantes:** Avecilla Tomás, Calcia Franco

---

## **Contexto**
Una persona se tomará 15 días de vacaciones en la playa. Durante al menos cuatro de esos días se espera mal clima, lo que podría limitar las actividades al aire libre. Se propone desarrollar una solución que, basada en el estado de ánimo del usuario, recomiende una actividad recreativa para realizar en interiores.

---

## **Objetivo**
Desarrollar un programa que recomiende entre ver una película, jugar un juego de mesa o leer un libro, basándose en el estado de ánimo y las preferencias del usuario.

---

## **Datasets Utilizados**
1. **bgg_database.csv:** Base de datos de juegos de mesa.
2. **IMDB-Movie-Data.csv:** Base de datos de películas.
3. **Libros del Proyecto Gutenberg:** Dataset obtenido mediante web scraping.


## Preparacion del entorno de trabajo

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install pandas
!pip install beautifulsoup4 requests
!pip install gdown
!pip install spacy
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 15.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 38.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import requests
import ast
from bs4 import BeautifulSoup
import torch
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import re
import unicodedata

Luego de importar los recursos necesarios descargamos los datsets disponibles

In [3]:
!gdown "1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq" --output "bgg_database.csv"
!gdown "1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c" --output "IMDB-Movie-Data.csv"

df_juegos = pd.read_csv('bgg_database.csv')

df_movies = pd.read_csv('IMDB-Movie-Data.csv')

Downloading...
From: https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
To: /content/bgg_database.csv
100% 1.83M/1.83M [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c
To: /content/IMDB-Movie-Data.csv
100% 309k/309k [00:00<00:00, 82.9MB/s]


### Dataset de libros
Para conformar el dataset de libros hicimos Web-Scraping


In [4]:
'''
# URL del Proyecto Gutenberg (ejemplo: top 1000 libros)
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Buscar todos los <li> dentro del <ol> de libros
    book_list = soup.find('ol').find_all('li')

    # Lista para almacenar los detalles de los libros
    books_details = []

    # Función para extraer el autor, resumen y géneros de cada libro
    def get_book_details(book_url):
        book_response = requests.get(book_url)
        if book_response.status_code == 200:
            book_soup = BeautifulSoup(book_response.content, 'html.parser')

            # Extraer el autor
            author_tag = book_soup.find('th', string="Author")
            author = author_tag.find_next_sibling('td').text.strip() if author_tag else "Unknown Author"

            # Extraer el resumen
            summary_tag = book_soup.find('th', string="Summary")
            summary = summary_tag.find_next_sibling('td').text.strip() if summary_tag else "No summary available"

            # Extraer los géneros (subjects)
            genres = []
            for subject_tag in book_soup.find_all('th', string="Subject"):
                genre = subject_tag.find_next_sibling('td').text.strip()
                genres.append(genre)

            return author, summary, ", ".join(genres)
        else:
            return "Unknown Author", "No summary available", "No genres available"

    # Iterar sobre cada libro, extraer detalles y guardarlos en una lista
    for book in book_list:
        title = book.text.strip()  # Título del libro
        link = book.find('a')['href']  # Enlace al libro
        full_link = f"https://www.gutenberg.org{link}"

        # Obtener los detalles del libro
        author, summary, genres = get_book_details(full_link)

        # Almacenar los detalles en la lista
        books_details.append({'Title': title, 'Author': author, 'Summary': summary, 'Genres': genres})

    # Convertir la lista de libros en un DataFrame
    df_libros = pd.DataFrame(books_details)

else:
    print(f"Error al acceder a la página: {response.status_code}")
'''

'\n# URL del Proyecto Gutenberg (ejemplo: top 1000 libros)\nurl = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"\n\n# Realizar la solicitud HTTP\nresponse = requests.get(url)\n\n# Verificar que la solicitud fue exitosa\nif response.status_code == 200:\n    soup = BeautifulSoup(response.content, \'html.parser\')\n\n    # Buscar todos los <li> dentro del <ol> de libros\n    book_list = soup.find(\'ol\').find_all(\'li\')\n\n    # Lista para almacenar los detalles de los libros\n    books_details = []\n\n    # Función para extraer el autor, resumen y géneros de cada libro\n    def get_book_details(book_url):\n        book_response = requests.get(book_url)\n        if book_response.status_code == 200:\n            book_soup = BeautifulSoup(book_response.content, \'html.parser\')\n\n            # Extraer el autor\n            author_tag = book_soup.find(\'th\', string="Author")\n            author = author_tag.find_next_sibling(\'td\').text.strip() if author_tag else "Unkn

In [5]:
# df_libros.to_csv('libros.csv', index=False)

In [6]:
!gdown "1YNoKUN7WJaTPfDIm8PVIpx55V5k1kLPb" --output "libros1.csv"
df_libros = pd.read_csv('libros1.csv')

Downloading...
From: https://drive.google.com/uc?id=1YNoKUN7WJaTPfDIm8PVIpx55V5k1kLPb
To: /content/libros1.csv
100% 1.22M/1.22M [00:00<00:00, 55.7MB/s]


## Preparacion de datasets


In [7]:
df_libros.info()
df_juegos.info()
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    1000 non-null   object
 1   Author   1000 non-null   object
 2   Summary  1000 non-null   object
 3   Genres   998 non-null    object
dtypes: object(4)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxp

Normalizamos las columnas importantes para futuros analisis. Ademas hacemos una copia para conservar los df originales

In [8]:
df_libros = df_libros.rename(columns={'Summary': 'Description'})
df_juegos = df_juegos.rename(columns={'description': 'Description'})
df_juegos = df_juegos.rename(columns={'game_name': 'Title'})


df_libros_og = df_libros.copy()
df_juegos_og = df_juegos.copy()
df_movies_og = df_movies.copy()

## Mejoramos la legibilidad de los generos

df_libros['genres_normalized'] = df_libros['Genres'].apply(lambda x: list(set([genre.strip().lower() for genre in x.split(',')])) if isinstance(x, str) else [])
df_movies['genres_normalized'] = df_movies['Genre'].apply(lambda x: list(set([genre.strip().lower() for genre in x.split(',')])))
df_juegos['genres_normalized'] = df_juegos['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])


In [9]:
# Cargar modelo de lenguaje y stopwords una sola vez
nlp_limpieza = spacy.load("en_core_web_md")
stopwords_en = nlp_limpieza.Defaults.stop_words

def limpiar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'\W+', ' ', texto)
    return texto

for df in [df_libros, df_juegos, df_movies]:
    df['Description'] = df['Description'].apply(limpiar_texto)
    df['genres_normalized'] = df['genres_normalized'].apply(lambda x: [genre for genre in x if genre not in stopwords_en])

## Clasificación del Estado de Ánimo

Elegimos **BERT** por su precisión y capacidad para capturar matices en el análisis de sentimientos, lo cual es crucial para identificar correctamente el estado de ánimo del usuario. Su estructura bidireccional permite comprender el contexto completo de cada palabra dentro de una frase, proporcionando una mejor clasificación emocional y adaptable a las categorías "Alegre", "Melancólico" y "Ni fu ni fa".

Además, **BERT** está optimizado para procesamiento en GPU, lo que mejora su rendimiento en Google Colab, acelerando las predicciones.

In [12]:
# Cargar el modelo NER de Spacy con gpu
spacy.require_gpu()

nlp_ner = spacy.load("en_core_web_trf")

# Cargar el modelo BERT para análisis de sentimientos
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
device = 0
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

def obtener_embedding(texto):
    inputs = tokenizer(texto, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.logits.cpu().numpy()

# Aplicar NER y generar embeddings
def procesar_descripcion(descripcion):
    # Aplicar NER
    doc = nlp_ner(descripcion)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]

    # Obtener embedding de la descripción
    embedding = obtener_embedding(descripcion)

    return embedding, entidades

# Generar embeddings y entidades para cada dataframe
df_libros['embedding'], df_libros['entidades'] = zip(*df_libros['Description'].apply(procesar_descripcion))
df_juegos['embedding'], df_juegos['entidades'] = zip(*df_juegos['Description'].apply(procesar_descripcion))
df_movies['embedding'], df_movies['entidades'] = zip(*df_movies['Description'].apply(procesar_descripcion))


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [14]:
def clasificar_sentimiento_descripcion(descripcion):
    # Tokenizar la descripción con truncamiento
    inputs = tokenizer(descripcion, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    # Clasificar el sentimiento
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = logits.argmax().item()

    # Clasificar el sentimiento basado en la clase predicha
    # Mapeamos el índice de clase a la etiqueta correspondiente
    if predicted_class in [0, 1]:  # Clases 1 y 2
        return "Melancólico"
    elif predicted_class == 2:  # Clase 3
        return "Ni fu ni fa"
    elif predicted_class in [3, 4]:  # Clases 4 y 5
        return "Alegre"

    # Opción para manejar el caso en que no se encuentre una clasificación
    return "Clasificación no reconocida"


# Aplicar la clasificación de sentimiento a cada dataframe
df_libros['sentiment'] = df_libros['Description'].apply(clasificar_sentimiento_descripcion)
df_juegos['sentiment'] = df_juegos['Description'].apply(clasificar_sentimiento_descripcion)
df_movies['sentiment'] = df_movies['Description'].apply(clasificar_sentimiento_descripcion)



Cargamos los modelos de traduccion debido a que el sistema esta diseñado para ser usado en español pero las bases de daos estan en ingles

In [10]:
# Cargar el modelo de traducción inglés a español
device = 0
traductor_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es", device=device)

# Cargar el modelo de traducción español a inglés
traductor_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en", device=device)

# Función para traducir descripciones
def traducir_texto(texto):
    traduccion = traductor_en_es(texto, max_length=512)
    return traduccion[0]['translation_text']

# Función para traducir la preferencia del usuario al inglés
def traducir_preferencia(preferencia):
    traduccion = traductor_es_en(preferencia, max_length=512)
    return traduccion[0]['translation_text']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Creamos los inputs haciendo uso de

```
IP Widgets
```



In [11]:
# Crear cuadros de texto para que el usuario ingrese su estado de ánimo y preferencia
input_emotion = widgets.Text(
    description="Sentimiento:",
    placeholder="¿Cómo te sientes hoy?",
    layout=widgets.Layout(width='50%')
)

input_preference = widgets.Text(
    description="Preferencia:",
    placeholder="¿Qué te gustaría hacer hoy?",
    layout=widgets.Layout(width='50%')
)

# Área de texto para mostrar las recomendaciones
output_textarea = widgets.Textarea(
    description="Recomendaciones:",
    layout=widgets.Layout(width='70%', height='250px'),
    placeholder="Aquí aparecerán tus recomendaciones basadas en tu estado de ánimo y preferencia.",
    disabled=True
)

# Botón que actuará como Enter
boton_analizar = widgets.Button(
    description="Obtener Recomendaciones",
    layout=widgets.Layout(width='auto')  # Ajustar el ancho automáticamente
)

## Flujo de enrtada

In [18]:
# Función para clasificar el estado de ánimo
def clasificar_sentimiento(label):
    if '1' in label or '2' in label:
        return "Melancólico"
    elif '3' in label:
        return "Ni fu ni fa"
    elif '4' in label or '5' in label:
        return "Alegre"

# Calcular la similitud en cada dataframe
def calcular_similitud(df, preferencia_embedding):
    # Calcular la similitud entre la preferencia y los embeddings del dataframe
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(preferencia_embedding, x)[0][0])
    return df

def verificar_coincidencias(df, preferencia_texto, sentimiento_usuario):
    # Verifica que 'entidades' esté en el DataFrame
    if 'entidades' not in df.columns:
        df['entidades'] = [[] for _ in range(len(df))]  # Columna vacía si falta

    # Extraer entidades de la preferencia del usuario
    entidades_preferencia = [ent.strip() for ent in nlp_ner(preferencia_texto).ents]

    # Crear una lista para almacenar las similitudes ajustadas
    adjusted_similarities = []

    # Recorrer cada fila del DataFrame
    for _, row in df.iterrows():
        # Verificar si hay coincidencias entre las entidades de la descripción y las de la preferencia
        matching_entities = any(ent in row['entidades'] for ent in entidades_preferencia)
        similarity = row['similarity']  # Obtener similitud antes del ajuste

        # Ajustar similitud en función de las coincidencias de entidades
        if matching_entities:
            similarity += 0.1  # Incremento si hay coincidencias

        # Incrementar similitud si el sentimiento coincide
        if row['sentiment'] == sentimiento_usuario:
            similarity += 0.1  # Ajuste por sentimiento

        # Asegurarse de que similarity no exceda 1
        similarity = min(similarity, .9999)

        # Agregar la similitud ajustada a la lista
        adjusted_similarities.append(similarity)

    # Asignar las similitudes ajustadas al DataFrame
    df['similarity'] = adjusted_similarities

    return df


def generar_recomendacion(preferencia_embedding, preferencia_texto):
    # Obtener el sentimiento del usuario con el pipeline de sentimiento
    result = sentiment_pipeline(preferencia_texto)[0]
    sentimiento_usuario = clasificar_sentimiento(result['label'])

    # Calcular similitud para cada dataframe
    df_libros_sim = calcular_similitud(df_libros, preferencia_embedding).copy()
    df_juegos_sim = calcular_similitud(df_juegos, preferencia_embedding).copy()
    df_movies_sim = calcular_similitud(df_movies, preferencia_embedding).copy()

    # Verificar coincidencias de entidades y ajustar similitud
    recomendaciones_libros = verificar_coincidencias(df_libros_sim, preferencia_texto, sentimiento_usuario).copy()
    recomendaciones_juegos = verificar_coincidencias(df_juegos_sim, preferencia_texto, sentimiento_usuario).copy()
    recomendaciones_peliculas = verificar_coincidencias(df_movies_sim, preferencia_texto, sentimiento_usuario).copy()

    # Añadir la columna 'tipo'
    recomendaciones_libros.loc[:, 'tipo'] = 'Libro'
    recomendaciones_juegos.loc[:, 'tipo'] = 'Juego'
    recomendaciones_peliculas.loc[:, 'tipo'] = 'Película'

    # Usar descripciones originales de los DataFrames _og
    recomendaciones_libros['Description'] = recomendaciones_libros['Title'].map(df_libros_og.groupby('Title')['Description'].first())
    recomendaciones_juegos['Description'] = recomendaciones_juegos['Title'].map(df_juegos_og.groupby('Title')['Description'].first())
    recomendaciones_peliculas['Description'] = recomendaciones_peliculas['Title'].map(df_movies_og.groupby('Title')['Description'].first())

    # Combinar y ordenar por similitud
    todas_recomendaciones = pd.concat([recomendaciones_libros, recomendaciones_juegos, recomendaciones_peliculas])
    mejores_recomendaciones = todas_recomendaciones.sort_values(by='similarity', ascending=False).head(3)

    # Formatear el output de recomendaciones
    recomendaciones_formateadas = (
        f"Recomendación Principal:\n1. {mejores_recomendaciones.iloc[0]['tipo']}: {mejores_recomendaciones.iloc[0]['Title']} - {traducir_texto(mejores_recomendaciones.iloc[0]['Description'])}\n\n"
        "Alternativas:\n"
        f"2. {mejores_recomendaciones.iloc[1]['tipo']}: {mejores_recomendaciones.iloc[1]['Title']} - {traducir_texto(mejores_recomendaciones.iloc[1]['Description'])}\n"
        f"3. {mejores_recomendaciones.iloc[2]['tipo']}: {mejores_recomendaciones.iloc[2]['Title']} - {traducir_texto(mejores_recomendaciones.iloc[2]['Description'])}"
    )

    return recomendaciones_formateadas


def on_button_click(b):
    frase_sentimiento = input_emotion.value
    frase_preferencia = input_preference.value

    if frase_sentimiento and frase_preferencia:
        # Obtener el sentimiento del usuario utilizando el pipeline de sentimiento directamente
        frase_sentimiento = traducir_preferencia(frase_sentimiento)
        # Analyze sentiment with BERT - THIS WAS CHANGED
        result = sentiment_pipeline(frase_sentimiento)[0]

        # Clasificar el sentimiento con la función existente
        sentimiento_usuario = clasificar_sentimiento(result['label'])

        # Traducir la preferencia al inglés
        preferencia_en_ingles = traducir_preferencia(frase_preferencia)

        # Obtener embedding de la preferencia en inglés y generar recomendaciones
        preferencia_embedding = obtener_embedding(preferencia_en_ingles)

        recomendaciones = generar_recomendacion(preferencia_embedding, frase_preferencia)

        # Mostrar la recomendación final
        output_textarea.value = (
            f"Estado de ánimo detectado: {sentimiento_usuario} (etiqueta del modelo: {result['label']})\n\n"
            f"{recomendaciones}"
        )
    else:
        output_textarea.value = "Por favor ingresa tanto el estado de ánimo como tu preferencia."


# Uso
boton_analizar.on_click(on_button_click)
display(input_emotion)
display(input_preference)
display(boton_analizar)
display(output_textarea)

Text(value='algo decaido', description='Sentimiento:', layout=Layout(width='50%'), placeholder='¿Cómo te sient…

Text(value='algo para distraer mi cabeza por mucho tiempo', description='Preferencia:', layout=Layout(width='5…

Button(description='Obtener Recomendaciones', layout=Layout(width='auto'), style=ButtonStyle())

Textarea(value='Estado de ánimo detectado: Ni fu ni fa (etiqueta del modelo: 3 stars)\n\nRecomendación Princip…